# Imports

In [ ]:
import pandas as pd 
import json
import re
from ast import literal_eval
import requests
import datetime
import openmeteo_requests
import requests_cache
from retry_requests import retry
import time

# Methods

In [ ]:
def camel_to_snake(camel_strings):
    snake_strings = []
    
    for string in camel_strings:
        temp = re.sub(r'(?<!^)(?=[A-Z])', '_', string).lower()
        snake_strings.append(temp)
        
    return snake_strings

# Importing Files

In [ ]:
with open('spotify_history/others/Marquee.json', 'r', encoding='utf-8') as f:
    marquee_json = json.load(f)
    
marquee = pd.json_normalize(marquee_json)

marquee.columns = camel_to_snake(marquee.columns)
marquee = marquee.drop_duplicates()

marquee.head()

In [ ]:
with open('spotify_history/others/Playlist1.json', 'r', encoding='utf-8') as f:
    playlist_json = json.load(f)

# 2. Normalize as usual
all_playlists = []
for i in playlist_json["playlists"]:
    i_playlist = pd.json_normalize(i["items"],sep='_')
    i_playlist["playlists_lastModifiedDate"] = i["lastModifiedDate"]
    i_playlist["playlists_name"] = i["name"]
    all_playlists.append(i_playlist)

playlist = pd.concat(all_playlists)
playlist.columns = camel_to_snake(playlist.columns)
playlist = playlist.drop_duplicates()
playlist.head()

In [ ]:
with open('spotify_history/others/YourLibrary.json', 'r', encoding='utf-8') as f:
    library_json = json.load(f)

# 2. Normalize as usual
library = pd.json_normalize(library_json, record_path=['tracks'], sep='_')

library.columns = camel_to_snake(library.columns)
library = library.drop_duplicates()

library.head()

In [ ]:
with open('spotify_history/music_history/Streaming_History_Audio_2017-2026.json', 'r', encoding='utf-8') as f:
    history_json = json.load(f)

# 2. Normalize as usual
history = pd.json_normalize(history_json, sep='_')

history.columns = camel_to_snake(history.columns)
history = history.drop_duplicates()

history.head()

# Creating the main table

In [ ]:
# joining history df with marquee df
final_df = pd.merge(history, marquee, how='left', left_on='master_metadata_album_artist_name', right_on='artist_name').drop(columns=['master_metadata_album_artist_name'])

# adding column in_playlist
final_df['in_playlist'] = final_df['master_metadata_track_name'].isin(playlist['track_track_name'])

# adding column in_library
final_df['in_library'] = final_df['master_metadata_track_name'].isin(library['track'])
final_df.to_csv("final_df.csv")
final_df